#RAG-система: 2 варианта

##Установка зависимостей

In [ ]:
!pip install sentence-transformers "transformers>=4.37.0" qdrant-client datasets pymorphy3 pymorphy3-dicts-ru

In [ ]:
!pip uninstall -y numpy scipy gensim

In [ ]:
!pip install numpy==1.23.5 scipy==1.10.1 gensim==4.3.2 transformers==4.37.0

##Необходимые импорты

In [ ]:
!pip list | grep numpy

In [ ]:
!pip install --force-reinstall scipy gensim transformers

In [ ]:
import gensim
import transformers
print(gensim.__version__)


In [ ]:
import numpy as np
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models
from nltk.stem.snowball import SnowballStemmer
import transformers.data.metrics.squad_metrics as squad_metrics
import pymorphy3
import gensim.downloader
import gensim
import math
import string
import re
random_state = 42
import shap

##Загрузка датасета, токенизатора и модели для генерации ответов

In [ ]:
dataset = load_dataset("bearberry/sberquadqa")["train"]

Берем первые 50 примеров из датасета:

In [ ]:
questions = dataset['question'][:50]
correct_answers = dataset['normalized_answers'][:50]
documents = dataset['context'][:50]

In [ ]:
device = "cuda"
model_name = "RefalMachine/RuadaptQwen2.5-1.5B-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model_to_generate = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Предположительно, для более точных ответов, нам нужно будет явно прописать в промпте, сколько слов должно быть сгенерировано. Поэтому сразу посмотрим, какое максимальное количество слов в эталонном ответе.

In [ ]:
max_len = 0
for answer in correct_answers:
  for example in answer:
    if len(example.split()) > max_len:
      max_len = len(example.split())
print(max_len) #такое у нас будет ограничение на количество генерируемых слов

##Ответная система без retrieval

**КОНТЕКСТ И ЧАНКИ:**

Первое, над чем нужно задуматься, - сколько подавать чанков в контекст для генерации ответа и как именно это делать. Если просто взять в качестве контекста весь документ (т.е. весь список в поле "context"), то генерация будет очень затратной и по времени, и по вычислительным ресурсам. Но в каждом списке для каждого чанка есть информация, релевантен он для ответа или нет, так что мы можем воспользоваться этой информацией и брать, например, только релевантные чанки для формирования контекста. Но, возможно, только релевантных чанков будет слишком мало для точного ответа на вопрос и понадобится больший контекст. В таком случае, можно взять, например, окно чанков размером 3/5/10 чанков (в середине окна всегда релевантный для ответа чанк) и посмотреть, как это повлияет на качество ответов. Формировать контекст так, чтобы релевантный чанк оказывался в середине окна, кажется логичным и оптимальным, потому что полезная для ответа информация может быть и до релевантного чанка и после. Напишем функции, которые будут формировать "окно контекстов" из чанков.

Еще одна эверистика, которая может помочь в улучшении качества, - просто повторить в промпте один и тот же релевантный чанк несколько раз. Для того, чтобы учесть это в функции, введем параметр rep: если rep == True, то повторяем один релевантный чанк window_size раз, если rep == False, то будет window_size разных чанков.

In [ ]:
def context_one_chunk(document):
    final_chunks = []
    for doc in document:
      if doc["is_relevant"]:
        final_chunks.append(doc["chunk"]) #релевантных чанков может быть несколько, объединяем их всех в один список
    return final_chunks

def context_chunks_window(document, rep, window_size):
    chunks, final_chunks = [], []
    if rep:
      chunks = context_one_chunk(document)
      final_chunks = [" ".join(chunks) for _ in range(window_size)]
    else:
      i_rev = None #позиция релевантного чанка
      for i, doc in enumerate(document):
          chunks.append(doc["chunk"])
          if doc["is_relevant"]:
              i_rev = i
      i_start = window_size//2
      i_final = math.ceil(window_size/2) #округляем вверх
      start = max(0, i_rev - i_start)
      end = min(len(chunks), i_rev + i_final)
      final_chunks = chunks[start:end]
    return final_chunks

**ПРОМПТ:**

Для лучших ответов промпт нужно сформулировать точно и лаконично, сделав акцент на характере ответа, который мы хотим получить. Проанализирова эталонные ответы, можно сказать, что для нас самое главное, чтобы ответ модели был как можно более точным и кратким и полностью следовал из контекста. Также, чтобы ответ не был слишком большим, можно эксплицитно указать на ограничение по словам. Еще одна важная деталь для формирования подходящего промпта: сначала задать контекст, а только потом вопрос, чтобы модель как бы "сосредоточилась" именно на контексте и брала информацию из него, а потом только из вопроса. Чтобы усилить эффект промпта, мы также пропишем в роли для системы, что модель - эксперт по кратким и точным ответам. Это должно помочь. Таким образом, финальный промпт должен выглядеть примерно так:



In [ ]:
prompt = (
    "Ответь максимально точно и кратко на вопрос, используя только данный контекст. "
    "Ответ не должен превышать 10 слов. Вопрос и контекст приведены ниже.\n\n"
    "КОНТЕКСТ: \n{}\n"
    "ВОПРОС: {}"
)

**ФУНКЦИЯ ДЛЯ ФОРМИРОВАНИЯ ЗАПРОСОВ ЯЗЫКОВОЙ МОДЕЛИ И ГЕНЕРАЦИИ ОТВЕТОВ:**

Для генерации новых токенов существует множество разных параметров, которые можно контролировать, это влияет на качество ответов модели и скорость генерации. Мы будем контролировать 2 параметра: **max_new_token** (максимальное количество новых токенов, которые модель может сгенерировать) и **num_beams** (количество лучей (beam search) при поиске наилучшего ответа; **Beam Search** исследует несколько возможных вариантов ответа и выбирает наиболее вероятный).

Если значение **max_new_token** маленькое, то ответ будет коротким, если большое, то может быть слишком длинным. Нам нужны короткие ответы, будем брать **маленькие значения** для этого параметра (10-20 токенов).

Чем больше **num_beams**, тем тщательнее модель подбирает ответ, но это замедляет генерацию. Оптимальное значение **3–5** (будет баланс между качеством и скоростью).

In [ ]:
def generate_response(questions, documents, rep, window_size, prompt, max_new_tokens, num_beams):
  responses = []
  for i, document in enumerate(documents):
    question = questions[i]
    final_chunks = context_chunks_window(document, rep, window_size)
    new_prompt = prompt.format('\n'.join(final_chunks), question)
    message  = [{"role": "system", "content": "Ты эксперт по кратким и точным ответам."}, #уточняем "роль" модели
                      {"role": "user", "content": new_prompt}]
    text = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True) #форматируем входные сообщения в нужный формат для LLM
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    if num_beams is None: #базовый случай, num_beams по умолчанию = 1
      generated_ids = model_to_generate.generate(model_inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=True)
    else: #с подбором параметра num_beams
      generated_ids = model_to_generate.generate(
            model_inputs.input_ids, max_new_tokens=max_new_tokens, num_beams=num_beams, early_stopping=True
        )
    response = tokenizer.batch_decode(generated_ids[:, model_inputs.input_ids.shape[-1]:], skip_special_tokens=True)[0]
    responses.append(response)
  return responses

Возьмем window_size = 10 (в части с ретривером это аналогично top_k, его мы тоже возьмем, равным 10), чтобы ответы были точнее (окно/top_k большего размера рассматривать не будем, т.к. генерация заметно замедлится).

In [ ]:
params = [
    {"rep": False, "max_new_tokens": 15, "num_beams": None, "window_size": 10},
    {"rep": True, "max_new_tokens": 15, "num_beams": 5, "window_size": 10},
    {"rep": False, "max_new_tokens": 15, "num_beams": 5, "window_size": 10}
]

#тестируем базовый случай (num_beams по умолчанию равно 1)
model_0 = generate_response(questions, documents, params[0]["rep"], params[0]["window_size"], prompt, params[0]["max_new_tokens"], params[0]["num_beams"])
model_0  #смотрим, что сгенерировалось

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
def model(x):
    responses = generate_response(x, [documents[0]], params[0]["rep"], params[0]["window_size"], prompt, params[0]["max_new_tokens"], params[0]["num_beams"])
    #if len(responses) != len(x):  # Если количество ответов не совпадает с количеством вопросов
    #    responses = [responses[0]] * len(x)  # Клонируем первый ответ для всех вопросов
    encoded_responses = label_encoder.fit_transform(responses)
    return np.array(responses).reshape(-1, 1)

explainer = shap.Explainer(model, tokenizer, max_new_tokens=512)
data = [questions[0]]
#data = questions
print(data)

In [ ]:
explanation = explainer(data)

In [ ]:
model_1 = generate_response(questions, documents, params[1]["rep"], params[1]["window_size"], prompt, params[1]["max_new_tokens"], params[1]["num_beams"])
model_1

In [ ]:
model_2 = generate_response(questions, documents, params[2]["rep"], params[1]["window_size"], prompt, params[2]["max_new_tokens"], params[2]["num_beams"])
model_2

##Нормализация ответов

Для дальнейшего сравнения реализуем три функции для нормализации: с морфологическим анализом (**лемматизация**), со **стеммингом** и с учетом **синонимизации**. С помощью синонимизации мы смягчим метрику и будем считать положительными те случаи, когда модель не сгенерировала слово, как в эталоне, но выдавала синоним этого слова.

In [ ]:
morph = pymorphy3.MorphAnalyzer()
stemmer = SnowballStemmer("russian")
w2v_model = gensim.downloader.load("word2vec-ruscorpora-300")

#Функция для получения синонимов слова с помощью модели word2vec
def get_synonym(word):
    if word in w2v_model:
        similar_words = w2v_model.most_similar(word, topn=5)
        for similar_word, score in similar_words:
            if score > 0.6:
                return similar_word
    return word

#Предобработка ответа
def process_response(response):
  response = response.lower()
  split_response = re.split(r'[.,!?;:()\[\]{}/-]', response)  #разделяем по знакам препинания
  response = [s.strip() for s in split_response if s.strip()] #убираем пустые строки, если они есть
  return response

#Нормализация с морфологическим анализом
def normalize_responses_morph(responses):
  norm_responses = []
  for response in responses:
    response = process_response(response)
    new_words = []
    for x in response:
        new_words += [morph.parse(word)[0].normal_form for word in x.split()]
    norm_responses.append(" ".join(new_words))
  return norm_responses

#Нормализация со стеммингом
def normalize_responses_stem(responses):
  norm_responses = []
  for response in responses:
    response = process_response(response)
    new_words = []
    for x in response:
        new_words += [stemmer.stem(word) for word in x.split()]
    norm_responses.append(" ".join(new_words))
  return norm_responses

#Нормализация с учетом ближайших векторов и синонимов
def normalize_with_vectors(responses):
    norm_responses = []
    new_words = normalize_responses_morph(responses)
    new_words = [get_synonym(word) for word in new_words]
    norm_responses.append(" ".join(new_words))
    return norm_responses

##Функция подсчета метрики, оценка результатов

In [ ]:
def compute_metrics_norm(norm_responses, norm_correct_answers):
    #считаем f-меру для каждой пары ответов и затем берем среднее значение
    return 100.0 * sum(squad_metrics.compute_f1(a, r) for a, r in zip(norm_correct_answers, norm_responses)) / len(norm_responses)

Для корректного сравнения и ответы модели, и эталонные ответы из датасета должны быть нормализованы одинаково, так что применим нормализацию к обоим спискам ответов:

In [ ]:
cor_answers = []
for answer in correct_answers:
  cor_answers.append(" ".join(x for x in answer)) #делаем из списка списков список строк

#нормализуем эталонные ответы
norm_cor_ans = [
    normalize_responses_morph(cor_answers),
    normalize_responses_stem(cor_answers),
    normalize_with_vectors(cor_answers)
    ]

In [ ]:
#нормализуем ответы моделей, выводим f-меру
normalizations = ["morph", "stem", "vectors"]
models_with_params = [model_0, model_1, model_2]
for model, param in zip(models_with_params, params):
  norm_pred_ans = [
      normalize_responses_morph(model),
      normalize_responses_stem(model),
      normalize_with_vectors(model)
  ]
  print(param)
  for normalization, norm_pred_answer, norm_cor_answer in zip(normalizations, norm_pred_ans, norm_cor_ans):
    print(f"F1 score with {normalization}: {compute_metrics_norm(norm_pred_answer, norm_cor_answer)}")
  print("")

Мы видим, что при увеличении количества лучей поиска качество ответов заметно улучшается, а также, что, если сравнивать не только нормализованные слова, но и прикрутить синонимизацию, то качество будет выше. Если не пользоваться синонимизацией, то можно опираться на морфологический анализ, с помощью него получается более высокая метрика. Также видно, что если просто повторять релевантные чанки в промпте, а не брать разные, то качество ответов будет лучше.

##Ответная система с retrieval

Будем использовать Qdrant в качестве базы данных для эмбеддингов (хорошо подходит, т.к. быстро ищет похожие документы, в ней можно хранить текст и метаданные, у нее простая интеграция с Python):

In [ ]:
#Подключение к локальной векторной БД
client = QdrantClient(":memory:")

#Генерируем эмбеддинги для документов
def generate_embeddings(docs, model):
    embeddings = []
    for chunk in docs:
        embedding = model.encode(chunk, convert_to_tensor=True)
        embeddings.append(embedding.cpu().numpy())
    return np.array(embeddings)

#Создаем векторную БД
def create_qdrant_collection(collection_name, vector_dim):
  client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
        size=vector_dim,
        distance=models.Distance.COSINE
            )
      )

#Загрузка данных в Qdrant
def store_embeddings_in_qdrant(embeddings, metadata, collection_name):
    points = []
    for i, embedding in enumerate(embeddings):
        point = models.PointStruct(
            id=i,
            vector=embedding.tolist(),
            payload=metadata[i]
        )
        points.append(point)
    batch_size = 100
    for start in range(0, len(points), batch_size):
        batch_points = points[start:start + batch_size]
        client.upsert(collection_name=collection_name, points=batch_points)

#Поиск наиболее близких к запросу документов, повторения возможны
def query_qdrant_rep(query_embedding, collection_name, top_k):
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=top_k,
        with_payload=True
    )
    return [hit.payload for hit in search_result]

#Поиск наиболее близких к запросу документов, документы не повторяются
def query_qdrant_no_rep(query_embedding, collection_name, top_k, exclude_ids=None):
    if exclude_ids is None:
        exclude_ids = set()

    query_filter = models.Filter(
        must_not=[models.HasIdCondition(has_id=list(exclude_ids))]
    ) if exclude_ids else None

    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=top_k,
        query_filter=query_filter,
    )

    retrieved_docs = [(hit.id, hit.payload["chunk"]) for hit in search_result]
    return retrieved_docs

#Функция для формировния контекстов из разных документов
def retrieve_dif_docs(query_embedding, collection_name, top_k):
    found_docs = set() #исключаем ID уже найденных докуметнов
    all_retrieved_docs = []

    for i in range(top_k):
        new_docs = query_qdrant_no_rep(query_embedding, collection_name, top_k, exclude_ids=found_docs)

        if not new_docs:
            break

        found_docs.update(doc[0] for doc in new_docs)  #запоминаем ID
        for doc in new_docs:
          if doc[1] not in all_retrieved_docs:
            all_retrieved_docs.append(doc[1])

    return all_retrieved_docs

Используем библиотеку SentenceTransformer, т.к. она разработана специально для векторизации предложений и поиска семантического сходства. В качестве модели для создания эмбеддингов возьмем **"intfloat/multilingual-e5-large"** (хорошо подходит, т.к. создает вектора, совместимые с Qdrant, поддерживает русский язык и подходит для RAG, т.к. обучена на задачах поиска).

In [ ]:
rmodel = SentenceTransformer("intfloat/multilingual-e5-large")

#Подготавливаем документы для векторизации
docs = []
for elem in documents:
  for chunk in elem:
    docs.append(chunk["chunk"])

#Подготавливаем метаданные, в этом словаре будут храниться сами тексты
metadata = [{'chunk': chunk} for chunk in docs]

collection_name = "RAG_vectors"

#Создадим БД
context_vectors = generate_embeddings(docs, model)
create_qdrant_collection(collection_name, context_vectors.shape[1])
store_embeddings_in_qdrant(context_vectors, metadata, collection_name)

In [ ]:
#Посмотрим на пример
top_k = 5
query_text = "чем представлены органические остатки?"
query_embedding = model.encode(query_text, convert_to_tensor=True).cpu().numpy()
result = query_qdrant_rep(query_embedding, collection_name, top_k) #документы могут повторяться
print("query result rep:", result)

result = retrieve_dif_docs(query_embedding, collection_name, top_k) #документы не могут повторяться
print("query result no rep:", result)

**ФУНКЦИЯ ДЛЯ ФОРМИРОВАНИЯ ЗАПРОСОВ ЯЗЫКОВОЙ МОДЕЛИ И ГЕНЕРАЦИИ ОТВЕТОВ:**

Принцип подбора значений параметров **max_new_tokens** и **num_beams** такой же, как и в части без retrieval.

In [ ]:
def generate_response_qdrant(questions, rep, collection_name, top_k, max_new_tokens, num_beams, prompt):
  responses = []
  for question in questions:
    query_embedding = model.encode(question, convert_to_tensor=True).cpu().numpy()
    if rep:
      relevant_metadata = query_qdrant_rep(query_embedding, collection_name, top_k)
      context = "\n".join([meta['chunk'] for meta in relevant_metadata])
    else:
      relevant_metadata = retrieve_dif_docs(query_embedding, collection_name, top_k)
      context = "\n".join(meta for meta in relevant_metadata)
    new_prompt = prompt.format(context, question)
    message  = [{"role": "system", "content": "Ты эксперт по кратким и точным ответам."},
                      {"role": "user", "content": new_prompt}]
    text = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    generated_ids = model_to_generate.generate(
          model_inputs.input_ids, max_new_tokens=max_new_tokens, num_beams=num_beams, early_stopping=True
      )
    response = tokenizer.batch_decode(generated_ids[:, model_inputs.input_ids.shape[-1]:], skip_special_tokens=True)[0]
    responses.append(response)
  return responses

In [ ]:
#базовую модель (num_beams = 1) тестировать не будем, будем сразу смотреть с num_beams = 5
params = [
    {"rep": True, "max_new_tokens": 15, "num_beams": 5, "top_k": 10},
    {"rep": True, "max_new_tokens": 20, "num_beams": 5, "top_k": 10},
    {"rep": False, "max_new_tokens": 20, "num_beams": 5, "top_k": 10}
]

prompt = (
    "Ответь максимально точно и кратко на вопрос, используя только данный контекст. "
    "Ответ не должен превышать 10 слов. Вопрос и контекст приведены ниже.\n\n"
    "КОНТЕКСТ: \n{}\n"
    "ВОПРОС: {}"
)
largemodel_0 = generate_response_qdrant(questions, params[0]["rep"], collection_name, params[0]["top_k"], params[0]["max_new_tokens"], params[0]["num_beams"], prompt)
largemodel_0

In [ ]:
largemodel_1 = generate_response_qdrant(questions, params[1]["rep"], collection_name, params[1]["top_k"], params[1]["max_new_tokens"], params[1]["num_beams"], prompt)
largemodel_1

In [ ]:
largemodel_2 = generate_response_qdrant(questions, params[2]["rep"], collection_name, params[2]["top_k"], params[2]["max_new_tokens"], params[2]["num_beams"], prompt)
largemodel_2

In [ ]:
#нормализуем ответы моделей, выводим f-меру
normalizations = ["morph", "stem", "vectors"]
models_with_params = [largemodel_0, largemodel_1, largemodel_2]
for model, param in zip(models_with_params, params):
  norm_pred_ans_q = [
      normalize_responses_morph(model),
      normalize_responses_stem(model),
      normalize_with_vectors(model)
  ]
  print(param)
  for normalization, norm_pred_answer, norm_cor_answer in zip(normalizations, norm_pred_ans_q, norm_cor_ans):
    print(f"F1 score with {normalization}: {compute_metrics_norm(norm_pred_answer, norm_cor_answer)}")
  print("")

Результаты примерно повторяют тенденции того, что мы видели в части без retrieval. Если брать одни и те же чанки для контекста, то качество увеличивается. Если взять макимальную длину генерируемых токенов немного побольше, то качество тоже может немного прирасти, это, видимо, связано с тем, что какие-то нужные слова не обрезаются, если генерируется больше токенов.

##Финальные заметки и выводы

В обеих частях удалось достичь неплохого качества (60-77%). В части без retrieval, когда были выбраны те же параметры, что и в части с retrieval, качество оказалось выше (77% vs 72%). Видимо, когда мы подаем модели чанки, которые релевантны согласно разметке в датасете, а не даем ретриверу “самому” найти релевантные чанки,  то ответы модели ближе к эталонным из датасета. Чтобы ответы модели были ближе к эталонным можно сделать следующие вещи:

1.   брать достаточно большое количество  чанков для формирования контекста (5-10 или, может быть, больше, если не так важна скорость генерации),
2.   контролировать максимальную длину генерируемых токенов (она должна быть не очень большой, т.к. ответы краткие),
3.   брать достаточное количество лучей для BeamSearch (5 оказалось оптимальным, можно было попробовать взять больше, но это бы замедлило генерацию),
4.   хорошо сработало повторение релевантных чанков в контексте, который
прописывается в промпте,
5.   если нам не так важно, чтобы модель генерировала именно те же корни и леммы, что и в эталонных ответах, мы можем смягчить подсчет метрик за счет нормализации с использованием синонимизации.